In [6]:
import numpy as np
import sympy as sym
from sympy import hermite, solveset, integrate, oo 
#Inciso a)
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

def GetHermite(n, x):
    if n == 0:
        return sym.simplify(1)
    elif n == 1:
        return sym.simplify(2*x)
    else:
        hermite_n_minus_1 = GetHermite(n - 1, x)
        hermite_n_minus_2 = GetHermite(n - 2, x)
        hermite = sym.simplify(2 * x * hermite_n_minus_1 - 2 * (n - 1) * hermite_n_minus_2)
        return hermite


def GetDHermite(n,x):
    Pn = GetHermite(n,x)
    return sym.diff(Pn,x,1)

def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn
    
def GetRootsGHer(f,df,x,tolerancia = 14):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)
    
        croot = np.round( root, tolerancia )
        
        if croot not in Roots:
            Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

def GetAllRootsGHer(n):

    xn = np.linspace(-np.sqrt(4*n+1),np.sqrt(4*n+1),100)
    
    Hermite = []
    DHermite = []
    
    for i in range(n+1):
        Hermite.append(GetHermite(i,x))
        DHermite.append(GetDHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n],'numpy')
    Dpoly = sym.lambdify([x],DHermite[n],'numpy')
    Roots = GetRootsGHer(poly,Dpoly,xn)
    
    return Roots
def GetRootsAllPoly (n):
  
    Roots=[]
    for i in range(1,n+1):
        Roots_n=GetAllRootsGHer(i)
        Roots.append(Roots_n)
        
    return Roots
raices_20_polinomios=GetRootsAllPoly(20)
#Inciso b)
## Si transformamos la expresión que se nos da, queda que la f(x) que multiplica a e^(-x^2) está dada por:
def funcion_armonico (x):
    return (x**3/(np.sqrt(np.pi)))
##Ya con la función, definimos los pesos de los polinomios e integramos
def GetWeightsGHer(n):

    Roots = GetAllRootsGHer(n) 
    Weights = []
   
    for i in range(len(Roots)):
        Hermite_n_minus_1 = GetHermite(n - 1, Roots[i])
        weight = ((2**(n-1))*(np.math.factorial(n-1))*(np.sqrt(np.pi)))/((n**2) * (np.abs(Hermite_n_minus_1)**2))
        Weights.append(weight)

    return Weights
def Integral (n):
    raices = GetAllRootsGHer(n)
    pesos = GetWeightsGHer(n)
    I = 0
    for i in range(n):
        I += pesos[i]*funcion_armonico(raices[i])
    return I
integral=Integral(3) 

KeyboardInterrupt: 